# :basketball: 1 - Create augmentations

## Imports

In [18]:
import cv2
import shutil
import albumentations as A

from pathlib import Path

In [19]:
import warnings
warnings.filterwarnings("ignore")

## Predefined variables

In [ ]:
TRAIN_DIR = Path("../data/basketball.yolov8/train")
TEST_DIR = Path("../data/basketball.yolov8/test")
VAL_DIR = Path("../data/basketball.yolov8/valid")
TRAIN_LABELS_DIR = Path("../data/basketball.yolov8/train/labels")
AUGMENTED_DIR = Path("../data/basketball.yolov8-augmented")
DATA_YAML = Path("../data/basketball.yolov8/data.yaml")

Path(AUGMENTED_DIR).mkdir(parents=True, exist_ok=True)

## Augmentations

In [21]:
augmentations = {
    "geometric": A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=15, p=0.5),
        A.RandomScale(scale_limit=0.2, p=0.5)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),

    "color": A.Compose([
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.5),
        A.RGBShift(p=0.5),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),

    "kernel": A.Compose([
        A.GaussianBlur(blur_limit=(1,3), p=0.5),
        A.MotionBlur(blur_limit=3, p=0.3)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
}

In [22]:
def create_augmented_dataset_with_original(category, transform):
    # Create augmented directories
    aug_train_img = AUGMENTED_DIR / category / "train/images"
    aug_train_lbl = AUGMENTED_DIR / category / "train/labels"
    aug_val = AUGMENTED_DIR / category / "valid"
    aug_test = AUGMENTED_DIR / category / "test"

    aug_train_img.mkdir(parents=True, exist_ok=True)
    aug_train_lbl.mkdir(parents=True, exist_ok=True)
    aug_val.mkdir(parents=True, exist_ok=True)
    aug_test.mkdir(parents=True, exist_ok=True)

    # Copy original training images and labels
    for img_file in (TRAIN_DIR / "images").glob("*.jpg"):
        shutil.copy(img_file, aug_train_img / img_file.name)
    for label_file in (TRAIN_LABELS_DIR).glob("*.txt"):
        shutil.copy(label_file, aug_train_lbl / label_file.name)

    # Generate augmented images and labels
    for img_file in (TRAIN_DIR / "images").glob("*.jpg"):
        img = cv2.imread(str(img_file))
        label_file = TRAIN_LABELS_DIR / f"{img_file.stem}.txt"

        # Read labels
        bboxes = []
        class_labels = []
        with open(label_file) as f:
            for line in f:
                cls, x, y, w, h = map(float, line.strip().split())
                bboxes.append([x, y, w, h])
                class_labels.append(int(cls))

        # Apply augmentation
        augmented = transform(image=img, bboxes=bboxes, class_labels=class_labels)
        img_aug = augmented['image']
        bboxes_aug = augmented['bboxes']

        # Save augmented image
        new_img_name = f"{img_file.stem}_{category}.jpg"
        cv2.imwrite(str(aug_train_img / new_img_name), img_aug)

        # Save labels
        new_label_name = f"{img_file.stem}_{category}.txt"
        with open(aug_train_lbl / new_label_name, "w") as f:
            for bbox, cls in zip(bboxes_aug, class_labels):
                f.write(f"{cls} {' '.join(map(str,bbox))}\n")

    # Copy validation and test sets, and data.yaml to augmented directory
    shutil.copytree(VAL_DIR, aug_val, dirs_exist_ok=True)
    shutil.copytree(TEST_DIR, aug_test, dirs_exist_ok=True)
    shutil.copy(DATA_YAML, AUGMENTED_DIR / category / "data.yaml")

In [23]:
for cat, aug in augmentations.items():
    create_augmented_dataset_with_original(cat, aug)

print("Augmented datasets created with original images included!")

Augmented datasets created with original images included!
